In [2]:
import pandas as pd

In [3]:
# Read the CSV files
cal = pd.read_csv('data/calendar2024.csv')
lis = pd.read_csv('data/listings2024.csv') 
rev = pd.read_csv('data/reviews2024.csv')

## TODO
- [ ] Many variables are stored as generic objects, instead of their actual datatype
- [ ] ETL: Extract, Transform, Load

---
> Attributes of datasets identified at initial look
- Primary keys: cal.listing_id, list.id, rev.listing_id


### Clean

In [4]:
cal.drop(columns=['adjusted_price'], inplace=True)

null_cols_lis = lis.columns[lis.isna().all()].tolist()
lis = lis.drop(columns=null_cols_lis)

lis.drop(columns=['scrape_id'], inplace=True)

rev.drop(columns=['reviewer_name'], inplace=True)

In [ ]:
cal['price'] = cal['price'].str.replace(r'[\$,]', '', regex=True)

cal = cal.rename(columns={'price': 'price($)'})
cal['price($)'] = pd.to_numeric(cal['price($)'], errors='coerce')

cal['available'] = cal['available'] == 't'
lis['instant_bookable'] = lis['instant_bookable'] == 't'

cal['date'] = pd.to_datetime(cal['date'])
rev['date'] = pd.to_datetime(rev['date'])
lis['last_scraped'] = pd.to_datetime(lis['last_scraped'])
lis['host_since'] = pd.to_datetime(lis['host_since'])

rev['comments'].astype("string")#(str)

In [ ]:
rev[:5]
# calendar_df[calendar_df['listing_id'] == 31094]

In [ ]:
# Convert to datetime
rev['comments'] = rev['comments'].astype("string")

# Check both type and dtype since astype(str) was called but comments may still not be string dtype
print("Python type:", type(rev['comments']))
print("Pandas dtype:", rev['comments'].dtype)
# rev.info()
# Sanity check
# print("Calendar date range:", cal['datetime'].min(), "to", cal['datetime'].max())

In [32]:
lis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20909 entries, 0 to 20908
Data columns (total 71 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            20909 non-null  int64  
 1   listing_url                                   20909 non-null  object 
 2   last_scraped                                  20909 non-null  object 
 3   source                                        20909 non-null  object 
 4   name                                          20909 non-null  object 
 5   description                                   20235 non-null  object 
 6   neighborhood_overview                         8984 non-null   object 
 7   picture_url                                   20909 non-null  object 
 8   host_id                                       20909 non-null  int64  
 9   host_url                                      20909 non-null 

In [33]:
print(lis.id)#.nunique())

0                      31094
1                     262961
2                     263036
3                      32379
4                     263708
                ...         
20904    1189068003466882100
20905    1188291058700847695
20906    1189086099059352914
20907    1188302911099164911
20908    1189116201603723600
Name: id, Length: 20909, dtype: int64


In [34]:
print(lis.instant_bookable)

0        False
1        False
2        False
3        False
4        False
         ...  
20904    False
20905    False
20906    False
20907    False
20908    False
Name: instant_bookable, Length: 20909, dtype: bool


In [ ]:
# Get columns with less than 10 unique values
low_unique_cols = [col for col in cal.columns if cal[col].nunique() < 3]
print("Columns with less than 3 unique values:")
for col in low_unique_cols:
    print(f"{col}: {cal[col].nunique()}")

In [ ]:
lis[lis.columns[10:20]].info()

In [35]:
lis.iloc[:2, 10:20]

,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url
0,Ebbe,2010-05-22,"Copenhagen, Denmark","Hi and welcome. My name is Ebbe, I am a medica...",NaN,NaN,NaN,f,https://a0.muscache.com/im/users/129976/profil...,https://a0.muscache.com/im/users/129976/profil...
1,Nina,2011-11-07,"Copenhagen, Denmark",Hej - I read the Master in Pedagogy. My husba...,NaN,NaN,100%,f,https://a0.muscache.com/im/pictures/user/43342...,https://a0.muscache.com/im/pictures/user/43342...


In [ ]:
rev[:5]

In [ ]:
lis.info()

In [ ]:
# remap to boolean + sanity check
# print(lis.groupby('instant_bookable').size())
# print(lis[['instant_bookable']].groupby('instant_bookable').head(2))

# lis['instant_bookable'] = lis['instant_bookable'] == 't'

# print(lis.groupby('instant_bookable').size())
# print(lis[['instant_bookable']].groupby('instant_bookable').head(2))

# print(rev.reviewer_name)#.nunique())
rev.describe()